In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from accelerate import Accelerator

In [2]:
x = torch.rand(5, 3)
print(x)

tensor([[0.8171, 0.6011, 0.4267],
        [0.0553, 0.5727, 0.2278],
        [0.2952, 0.0677, 0.4278],
        [0.6590, 0.4088, 0.5218],
        [0.0394, 0.5800, 0.8707]])


In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:07<00:00, 3.43MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 186kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.20MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 2.50MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [10]:
batch_size=64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
    print(f'Shape of X [N,C,H,W]:{X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break

Shape of X [N,C,H,W]:torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [18]:
accelerator = Accelerator()
device = accelerator.device
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch%100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [22]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)
            pred = model(X)
            test_loss+= loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f}\n')


In [23]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n -----------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done')

Epoch 1
 -----------
loss: 2.309705 [   64/60000]
loss: 2.296659 [ 6464/60000]
loss: 2.272269 [12864/60000]
loss: 2.273092 [19264/60000]
loss: 2.249103 [25664/60000]
loss: 2.215642 [32064/60000]
loss: 2.232983 [38464/60000]
loss: 2.190335 [44864/60000]
loss: 2.184555 [51264/60000]
loss: 2.153526 [57664/60000]
Test error: 
 Accuracy: 33.5%, Avg loss:2.148007

Epoch 2
 -----------
loss: 2.156217 [   64/60000]
loss: 2.150253 [ 6464/60000]
loss: 2.079691 [12864/60000]
loss: 2.107744 [19264/60000]
loss: 2.058796 [25664/60000]
loss: 1.985877 [32064/60000]
loss: 2.029455 [38464/60000]
loss: 1.935523 [44864/60000]
loss: 1.941703 [51264/60000]
loss: 1.879011 [57664/60000]
Test error: 
 Accuracy: 50.9%, Avg loss:1.869767

Epoch 3
 -----------
loss: 1.896890 [   64/60000]
loss: 1.875993 [ 6464/60000]
loss: 1.741672 [12864/60000]
loss: 1.801815 [19264/60000]
loss: 1.692737 [25664/60000]
loss: 1.636725 [32064/60000]
loss: 1.674239 [38464/60000]
loss: 1.564340 [44864/60000]
loss: 1.595793 [51264/600

In [24]:
torch.save(model.state_dict(),"demo_model.pth")
print('Saving model to demo_model.pth')


Saving model to demo_model.pth


In [25]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("demo_model.pth", weights_only=True))

<All keys matched successfully>

In [26]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)],classes[y]
    print(f'Predicted: {predicted}, Actual: {actual}')

Predicted: Ankle boot, Actual: Ankle boot
